<h1 style="text-align: center;">Práctica 4: BLAST </h1>
</h3>
<h3 style="text-align: center;"> Gisela Belmonte Cruz; Diego Marrero Ferrera </h3>


En esta práctica trabajaremos con BLAST (Basic Local Alignment Search Tool), una herramienta fundamental en bioinformática que permite comparar una secuencia biológica con millones de secuencias almacenadas en bases de datos internacionales.

Nuestro objetivo es aprender a usar BLAST online y BLAST local, además de procesar sus resultados utilizando Biopython. Para los recursos utilizados se han usado los archivos y conexiones que ofrece [NCBI](https://blast.ncbi.nlm.nih.gov/Blast.cgi).

Cada variante del algoritmo permite comparar secuencias de ADN, ARN o proteínas para identificar genes homólogos, funciones posibles o relaciones evolutivas. La práctica nos introduce a:

- Ejecutar BLAST desde Python,
- Procesar salidas en formato XML,
- Analizar resultados (E-value, identidad, etc.),
- Y trabajar con bases de datos locales o remotas.

---

## Ejercicio 1: BLAST a partir de una secuencia de ADN introducida por teclado.

En el Ejercicio 1 se nos pide realizar una búsqueda BLAST de ADN, introducida por teclado o leída desde un archivo FASTA, mostrando tres resultados:
1. Número total de alineamientos obtenidos
2. El E-value del mejor resultado
3. La descripción de la secuencia más similar

Trabajaremos tanto en versión online como local.

Para este ejercicio utilizaremos el archivo `rna.fna`, que contiene la secuencia de **ARNm del gen de la oxitocina humana**. Elegimos este archivo porque:
- Representa la versión ya procesada del gen,
- Es más corto y limpio que la secuencia genómica (gene.fna),
- Y es ideal para realizar una consulta `blastn`.

A continuación, cargaremos esa secuencia y ejecutaremos BLAST de manera online utilizando los servidores del NCBI.

In [1]:
from Bio import SeqIO

def leer_secuencias_rna(archivo):
    # Leer todas las secuencias del archivo
    registros = list(SeqIO.parse(archivo, "fasta"))
    print("Número de secuencias encontradas:", len(registros))

    # Mostramos la info de cada una
    for i, r in enumerate(registros):
        print(f"{i}: {r.id} (longitud: {len(r.seq)})")

    # Elegimos la primera secuencia porque es la más completa
    registro = registros[0]
    secuencia_query = str(registro.seq)

    print("\nSecuencia seleccionada:")
    print("ID:", registro.id)
    print("Longitud:", len(secuencia_query))
    print("Secuencia:", secuencia_query)
    return secuencia_query

secuencia_query = leer_secuencias_rna("./local/rna_oxi.fna")

Número de secuencias encontradas: 6
0: NM_000916.4 (longitud: 4387)
1: NM_001354653.2 (longitud: 4307)
2: NM_001354654.2 (longitud: 4364)
3: NM_001354655.2 (longitud: 4189)
4: NM_001354656.3 (longitud: 4048)
5: XR_007095681.1 (longitud: 3218)

Secuencia seleccionada:
ID: NM_000916.4
Longitud: 4387
Secuencia: GTTAAGGCTCTGGGACCAACGCTGGGCGAACCAGCTCCGCTCCGGAGGGGTCTGCGCGGCTGGCCTCGCCCGCCCCCTAGCGGACCCGTGCGATAGTGCAGCCTCAGCCCCAGCGCACAGCGCCGCATCCAGACGCTGTCCGCGCGCGCAGCCTGGGAGGCGCTCCTCGCTCGCCTCCTGTACCCATCCAGCGACCAGCCAGGCTGCGGCGAGGGGATTCCAACCGAGGCTCCAGTGAGAGACCTCAGCTTAGCATCACATTAGGTGCAGCCGGCAGGCCATCCCAACTCGGGCCGGGAGCGCACGCGTCACTGGGGCCGTCAGTCGCCGTGCAACTTCCCCGGGGGGAGTCAACTTTAGGTTCGCCTGCGGACTCGGTGCAGTGGAAGCCGCTGAACATCCCGAGGAACTGGCACGCTGGGGGCTCTGGGCTTGTGGCCGGTAGAGGATTCCCGCTCATTTGCAGTGGCTCAGAGGAGGGTGGACCCAGCAGATCCGTCCGTGGAGTCTCCAGGAGTGGAGCCCCGGGCGCCCCTACACCCTCCGACACGCCGGATCCGGCCCAGCCGCGCCAAGCCGTAAAGGGCTCGAAGGCCGGGGCGCACCGCTGCCGCCAGGGTCATGGAGGGCGCGCTCGCAGCCAACTGGAGCGCCGAGGCAGCCAACGCCAGCGCCGCGCCGCCGGGGGCC

Sabiendo la secuencia a buscar, ejecutamos BLAST online en NCBI:

In [2]:
from Bio.Blast import NCBIWWW

print("Realizando búsqueda BLAST (blastn)...")

# Descomentar esto para ejecutar BLAST en línea (requiere conexión a internet y bastante tiempo de ejecución)
# ---
# resultado_xml = NCBIWWW.qblast(
#     program="blastn",
#     database="nt",
#     sequence=secuencia_query
# )

# # Guardar en archivo, en la carpeta "xml_creados"
# with open("./xml_creados/resultado_ej1_igf.xml", "w") as f:
#     f.write(resultado_xml.read())
# ---

print("BLAST completado. Archivo guardado como resultado_ej1_oxitocina.xml")


Realizando búsqueda BLAST (blastn)...
BLAST completado. Archivo guardado como resultado_ej1_oxitocina.xml


Con el archivo XML que obtuvimos, lo procesamos y podemos mostrar los resultados solicitados:

In [3]:
from Bio.Blast import NCBIXML

def analizar_resultados_blast(archivo_xml):
    with open(archivo_xml) as f:
        blast_records = list(NCBIXML.parse(f))

    record = blast_records[0]

    print("Número total de alineamientos con E-value < 1e-10 (Alineamientos casi o totalmente perfectos):", sum(1 for a in record.alignments if a.hsps[0].expect < 1e-10))
    print("-------------------------------")
    # Mejor alineamiento:
    best = record.alignments[0]
    best_hsp = best.hsps[0]

    print("=== MEJOR RESULTADO ===")
    print("Descripción:", best.hit_def)
    print("E-value:", best_hsp.expect)
    print("Longitud del alineamiento:", best.length)
    print("-------------------------------")
    print("Podio de mejores alineamientos:")
    for i, alignment in enumerate(record.alignments[:5]):
        print(f"  {i + 1}. {alignment.hit_def} (E-value: {alignment.hsps[0].expect})")
    print("-------------------------------")
    return record

record = analizar_resultados_blast("./xml_creados/resultado_ej1_oxitocina.xml")


Número total de alineamientos con E-value < 1e-10 (Alineamientos casi o totalmente perfectos): 50
-------------------------------
=== MEJOR RESULTADO ===
Descripción: Homo sapiens oxytocin receptor (OXTR), transcript variant 1, mRNA
E-value: 0.0
Longitud del alineamiento: 4387
-------------------------------
Podio de mejores alineamientos:
  1. Homo sapiens oxytocin receptor (OXTR), transcript variant 1, mRNA (E-value: 0.0)
  2. Homo sapiens oxytocin receptor (OXTR), transcript variant 3, mRNA (E-value: 0.0)
  3. PREDICTED: Pan troglodytes oxytocin receptor (OXTR), transcript variant X1, mRNA (E-value: 0.0)
  4. PREDICTED: Gorilla gorilla gorilla oxytocin receptor (OXTR), transcript variant X1, mRNA (E-value: 0.0)
  5. Homo sapiens oxytocin receptor (OXTR), transcript variant 2, mRNA (E-value: 0.0)
-------------------------------


Tenemos un gran número de coincidencias exactas o casi exactas en el caso de la oxitocina. Para probar con otro ejemplo, vamos a usar el gen del **factor de crecimiento de la insulina (Insulin Growth Factor)** para realizar el mismo procedimiento y comparar resultados:

In [4]:
secuencia_query_ferr = leer_secuencias_rna("./local/rna_igf.fna")
print("Realizando búsqueda BLAST (blastn) para IGF...")
# Descomentar esto para ejecutar BLAST en línea (requiere conexión a internet y bastante tiempo de ejecución)
# ---
# resultado_xml_ferr = NCBIWWW.qblast("blastn", "nt", secuencia_query_ferr)
# with open("./xml_creados/resultado_ej1_igf.xml", "w") as f:
#     f.write(resultado_xml_ferr.read())
# ---

print("BLAST completado. Archivo guardado como resultado_ej1_igf.xml")

Número de secuencias encontradas: 13
0: NM_000618.5 (longitud: 7277)
1: NM_001111283.3 (longitud: 7326)
2: NM_001111284.2 (longitud: 7096)
3: NM_001111285.3 (longitud: 1097)
4: NM_001414005.1 (longitud: 979)
5: NM_001414006.1 (longitud: 916)
6: NM_001414007.1 (longitud: 7159)
7: XM_017019259.2 (longitud: 989)
8: XM_017019262.3 (longitud: 7218)
9: XM_017019263.3 (longitud: 7169)
10: XM_054371951.1 (longitud: 993)
11: XM_054371952.1 (longitud: 7224)
12: XM_054371953.1 (longitud: 7175)

Secuencia seleccionada:
ID: NM_000618.5
Longitud: 7277
Secuencia: CTTCTGTTTGCTAAATCTCACTGTCACTGCTAAATTCAGAGCAGATAGAGCCTGCGCAATGGAATAAAGTCCTCAAAATTGAAATGTGACATTGCTCTCAACATCTCCCATCTCTCTGGATTTCTTTTTGCTTCATTATTCCTGCTAACCAATTCATTTTCAGACTTTGTACTTCAGAAGCAATGGGAAAAATCAGCAGTCTTCCAACCCAATTATTTAAGTGCTGCTTTTGTGATTTCTTGAAGGTGAAGATGCACACCATGTCCTCCTCGCATCTCTTCTACCTGGCGCTGTGCCTGCTCACCTTCACCAGCTCTGCCACGGCTGGACCGGAGACGCTCTGCGGGGCTGAGCTGGTGGATGCTCTTCAGTTCGTGTGTGGAGACAGGGGCTTTTATTTCAACAAGCCCACAGGGTATGGCTCCAGCAGTCGGAGGGCGCCTCA

In [5]:
# Y ahora, analizamos los resultados para el IGF
record_igf = analizar_resultados_blast("./xml_creados/resultado_ej1_igf.xml")

Número total de alineamientos con E-value < 1e-10 (Alineamientos casi o totalmente perfectos): 50
-------------------------------
=== MEJOR RESULTADO ===
Descripción: Homo sapiens insulin like growth factor 1 (IGF1), transcript variant 4, mRNA
E-value: 0.0
Longitud del alineamiento: 7277
-------------------------------
Podio de mejores alineamientos:
  1. Homo sapiens insulin like growth factor 1 (IGF1), transcript variant 4, mRNA (E-value: 0.0)
  2. Homo sapiens insulin like growth factor 1 (IGF1), transcript variant 1, mRNA (E-value: 0.0)
  3. Human IGF-I mRNA for insulin-like growth factor I (E-value: 0.0)
  4. PREDICTED: Pan troglodytes insulin like growth factor 1 (IGF1), transcript variant X7, mRNA (E-value: 0.0)
  5. Homo sapiens insulin like growth factor 1 (IGF1), transcript variant 7, mRNA (E-value: 0.0)
-------------------------------


## Ejercicio 2

En este ejercicio realizamos una búsqueda **BLAST de proteína** utilizando Biopython.
A partir del archivo `protein_oxi.faa` descargado desde NCBI, identificamos cinco secuencias correspondientes a diferentes isoformas del producto proteico del gen **OXT (oxitocin/neurophysin I precursor)** en *Homo sapiens*.

Todas ellas tienen longitud 389 aminoácidos, lo cual confirma que son variantes anotadas del precursor proteico de la oxitocina.

Para la práctica seleccionaremos:

* **ID:** NP_000907.2
* **Tipo:** RefSeq (proteína principal)
* **Longitud:** 389 aa

El objetivo del ejercicio es:

1. Realizar un **blastp** online contra la base de datos `nr`
2. Filtrar hits con **E-value < 0.001**
3. Guardarlos en un fichero con:
   * Identificador
   * Longitud
   * E-value
   * Porcentaje de identidad

In [6]:
# Leer todas las proteínas del archivo protein.faa
proteinas = list(SeqIO.parse("./local/protein_oxi.faa", "fasta"))

# Seleccionar la principal (NP_000907.2)
prot = proteinas[0]
seq_prot = str(prot.seq)

print("Secuencia seleccionada:")
print("ID:", prot.id)
print("Longitud:", len(seq_prot))


Secuencia seleccionada:
ID: NP_000907.2
Longitud: 389


In [7]:
print("Ejecutando blastp...")

# Descomentar esto para ejecutar BLASTP en línea (requiere conexión a internet y bastante tiempo de ejecución)
# ---
# resultado_xml = NCBIWWW.qblast(
#     program="blastp",
#     database="nr",
#     sequence=seq_prot
# )

# with open("./xml_creados/resultado_ej2_oxi.xml", "w") as f:
#     f.write(resultado_xml.read())
# ---

print("BLASTP completado. Archivo resultado_ej2_oxi.xml generado.")


Ejecutando blastp...
BLASTP completado. Archivo resultado_ej2_oxi.xml generado.


In [8]:
with open("./xml_creados/resultado_ej2_oxi.xml") as f:
    blast_records = list(NCBIXML.parse(f))

record = blast_records[0]

salida = []

for aln in record.alignments:
    for hsp in aln.hsps:
        if hsp.expect < 0.001:
            identidad = (hsp.identities / hsp.align_length) * 100
            salida.append({
                "id": aln.hit_id,
                "descripcion": aln.hit_def,
                "longitud": aln.length,
                "evalue": hsp.expect,
                "identidad": identidad
            })

# Guardar resultados
with open("./resultados_txt/resultado_filtrado_ej2_oxi.txt", "w") as f:
    for s in salida:
        f.write(f"{s['id']}\t{s['longitud']}\t{s['evalue']}\t{round(s['identidad'],2)}%\t{s['descripcion']}\n")

print("Resultados filtrados guardados en /resultados_txt/resultado_filtrado_ej2_oxi.txt")


Resultados filtrados guardados en /resultados_txt/resultado_filtrado_ej2_oxi.txt


Con el archivo ya guardado, vamos a ver las mayores coincidencias que encontramos en el archivo exportado:

In [9]:
# Mostramos los resultados de las mejores coincidencias del archivo txt exportado:
with open("./resultados_txt/resultado_filtrado_ej2_oxi.txt") as f:
    for line in range(10):
        print(f.readline().strip())

pdb|7RYC|O	450	0.0	100.0%	Chain O, Oxytocin receptor [Homo sapiens]
ref|NP_000907.2|	389	0.0	100.0%	oxytocin receptor [Homo sapiens] >ref|NP_001341582.1| oxytocin receptor [Homo sapiens] >ref|NP_001341583.1| oxytocin receptor [Homo sapiens] >ref|NP_001341584.1| oxytocin receptor [Homo sapiens] >ref|NP_001341585.1| oxytocin receptor [Homo sapiens] >sp|P30559.2| RecName: Full=Oxytocin receptor; Short=OT-R [Homo sapiens] >gb|AIC54855.1| OXTR, partial [synthetic construct] >emb|SJX46282.1| unnamed protein product, partial [Human ORFeome Gateway entry vector] >gb|AAI37444.1| Oxytocin receptor [Homo sapiens] >gb|EAW63947.1| oxytocin receptor [Homo sapiens] >gb|KAI4028104.1| oxytocin receptor [Homo sapiens]
gb|AAQ91333.1|	389	0.0	99.74%	oxytocin receptor [Homo sapiens] >gb|KAI2528039.1| oxytocin receptor [Homo sapiens] >emb|CAA46097.1| oxytocin receptor [Homo sapiens]
ref|XP_055238791.1|	389	0.0	99.23%	oxytocin receptor [Gorilla gorilla gorilla] >ref|XP_063559638.1| oxytocin receptor [Gorilla

#### 1. Secuencias humanas:
Como se puede apreciar, en las primeras dos filas tenemos los mejores alineamientos con una exactitud del 100%, esto indica que hay:
- Coincidencia exacta con la estructura cristalográfica del oxytocin receptor humano.
- Coincidencia con la proteína RefSeq humana (varias isoformas del mismo receptor). 

Esto es evidente, pues nuestra secuencia es humana por lo que los hits más parecidos son secuencias humanas idénticas.

#### 2. Primates cercanos:
Por otro lado, son muy interesantes las consecuentes similitudes como los más parecidos que corresponden a:
- Gorila (Gorilla gorilla gorilla): 99.23%
- Chimpancé (Pan troglodytes, Pan paniscus): 98.97%
- Gibbon (Symphalangus syndactylus, Nomascus leucogenys): 98.97%
- Orangután (Pongo abelii, Pongo pygmaeus): 98.2%

Esto refleja la cercanía evolutiva entre primates y humanos.

#### 3. Monos del Viejo Mundo y del Nuevo Mundo:
- Macaca mulatta (mono rhesus): 97.4%
- Papio anubis (babuino): 97.17%
- Aotus azarai (mono nocturno): ~95%

Cuanto **más lejana la especie, más identidad disminuye**, pero aún conservan un grado alto debido a la función esencial del receptor de oxitocina en mamíferos sociales.

#### 4. Mamíferos más alejados (90-94%):
Ejemplos:
- Castor (Castor canadensis): ~92%
- Rinoceronte (Diceros bicornis): ~91%
- Marmota marmota: ~93%

Aunque están lejos evolutivamente, mantienen la estructura general del receptor, porque son animales cercanos (grandes mamíferos) de los cuales además son conocidos como especies muy inteligentes que además comparten un estilo de vida o de estructura muy similar a la del humano de ir en manada, o "*sociedad*" en nuestro caso, siendo conocidos por también formar vínculos afectivos fuertes entre ellos como hacemos nosotros.

## Ejercicio 3 — BLAST de ADN desde archivo FASTA con filtrado por organismo

En este ejercicio utilizaremos la secuencia genómica completa del gen **OXT (oxytocin)** contenida en el archivo `gene_oxi.fna`.
Este archivo tiene dos secuencias, correspondientes a anotaciones cromosómicas distintas.
Seleccionaremos la primera:

* **ID:** NC_000003.12:c8769613-8741269
* **Longitud:** 28.345 bp
* **Organismo:** *Homo sapiens*

Realizaremos un **blastn** de esta secuencia contra la base `nt` y filtraremos los hits por un organismo elegido (por ejemplo: “Homo sapiens”, “Gorilla gorilla”, etc.).
Finalmente, mostraremos:

* Número de resultados del organismo seleccionado
* E-value medio de esos resultados


In [10]:
# Leer todas las secuencias del archivo gene_oxi.fna
genes = list(SeqIO.parse("./local/gene_oxi.fna", "fasta"))

# Seleccionar la primera secuencia
registro_gene = genes[0]
seq_gene = str(registro_gene.seq)

print("Secuencia seleccionada:")
print("ID:", registro_gene.id)
print("Longitud:", len(seq_gene))


Secuencia seleccionada:
ID: NC_000003.12:c8769613-8741269
Longitud: 28345


In [11]:
print("Ejecutando BLASTN...")

# Descomentar esto para ejecutar BLASTN en línea (requiere conexión a internet y bastante tiempo de ejecución)
# ---
# resultado_gene_xml = NCBIWWW.qblast(
#     program="blastn",
#     database="nt",
#     sequence=seq_gene
# )

# with open("./xml_creados/resultado_ej3_oxi.xml", "w") as f:
#     f.write(resultado_gene_xml.read())
# ---

print("BLAST completado. Archivo resultado_ej3_oxi.xml generado.")


Ejecutando BLASTN...
BLAST completado. Archivo resultado_ej3_oxi.xml generado.


A continuación elegimos el organismo para filtrar los resultados por organismo y calcular estadísticas:

In [12]:
organismo_objetivo = "Homo sapiens"  

In [13]:
import numpy as np
import pandas as pd

# Cargar el XML de BLAST
with open("./xml_creados/resultado_ej3_oxi.xml") as f:
    blast_records = list(NCBIXML.parse(f))
record = blast_records[0]

def analizar_organismo(record, nombre):
    """
    Filtra los alineamientos por organismo e informa del número de hits
    y el E-value medio.
    """
    hits = []
    for aln in record.alignments:
        if nombre.lower() in aln.hit_def.lower():
            for hsp in aln.hsps:
                hits.append(hsp.expect)
    if hits:
        return len(hits), np.mean(hits)
    else:
        return 0, None


In [14]:
organismos = [
    "Homo sapiens",
    "Pan troglodytes",      # chimpancé
    "Macaca mulatta",       # macaco rhesus
    "Papio anubis",         # babuino
    "Rhinopithecus roxellana"     # langur (mono pequeño de China)
]

resultados = []

for org in organismos:
    num_hits, evalue_medio = analizar_organismo(record, org)
    resultados.append({
        "Organismo": org,
        "Hits encontrados": num_hits,
        "E-value medio": evalue_medio
    })

df_resultados = pd.DataFrame(resultados)

df_resultados


,Organismo,Hits encontrados,E-value medio
0,Homo sapiens,124,2.247521e-02
1,Pan troglodytes,7,4.216586e-37
2,Macaca mulatta,4,3.137650e-35
3,Papio anubis,3,8.577300e-158
4,Rhinopithecus roxellana,21,6.392381e-04


Frente a estos resultados, vemos:

#### Respecto al Homo Sapiens

Vemos muchos _hits_ del homo sapiens, como es de esperar, y otros hits en especies que seleccionamos de otros primates. 
Aunque el e-valor promedio del homo sapiens sea mayor, lo que es paradójico a primera vista, se debe a que: 

__Muchos más hits = mucha diversidad = peores E-values en algunos alineamientos__

Los 124 hits incluyen:
- Isoformas
- Transcritos parciales
- Regiones cercanas del cromosoma asociado a nuestra secuencia
- Secuencias duplicadas
- Fragmentos con poca cobertura
- Artefactos o pseudogenes anotados

Esto incluye hits excelentes (E-value 0 o cerca de 0), pero también muchos hits peores, y por tanto la media se dispara.

#### Otras especies

En cambio, en otras especies solo aparecen hits "limpios"

Por ejemplo:

- Chimpancé (Pan troglodytes): solo 7 hits -> todos verdaderos ortólogos -> E-value medio bajísimo.
- Papio anubis: solo 3 hits, pero extremadamente conservados -> E-value medio de $10^{-158}$ (Sorprendentemente ESPECTACULAR).
- Macaca mulatta: 4 hits altamente conservados.

__Pocas coincidencias, pero muy fuertes.__ Esto da medias bajísimas.

## Ejercicio 4 - Comparación de las cinco variantes de la suite BLAST

En este ejercicio utilizaremos las cinco herramientas principales de BLAST:

| Herramienta | Query                    | Base de datos | Útil para                                           |
| ----------- | ------------------------ | ------------- | --------------------------------------------------- |
| **blastn**  | ADN                      | ADN           | Comparar secuencias genómicas o cDNA                |
| **blastp**  | proteína                 | proteína      | Homólogos proteicos                                 |
| **blastx**  | ADN traducido (6 marcos) | proteína      | Encontrar posibles proteínas codificadas por un gen |
| **tblastn** | proteína                 | ADN traducido | Buscar genes en genomas                             |
| **tblastx** | ADN traducido            | ADN traducido | Comparación muy sensible, pero costosa              |


Usaremos las secuencias:
- gene_oxi.fna -> para blastn, blastx y tblastx
- protein.faa -> para blastp y tblastn

Nuestro objetivo es:
1. Ejecutar las 5 herramientas
2. Guardar los resultados
3. Comparar:
   - qué especies aparecen primero
   - identidad
   - cobertura
   - tipo de coincidencias